In [2]:
import pandas as pd 
import numpy as np
import os 
from sklearn.cluster import KMeans,AgglomerativeClustering,DBSCAN,OPTICS,MeanShift
import os, shutil, glob, os.path
from PIL import Image as pil_image
from matplotlib import pyplot as plt

import sklearn
from sklearn.feature_extraction.image import grid_to_graph
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
#importing the header files 
import os
import numpy as np
import pandas as pd
import IPython.display as ipd
from sklearn.manifold import TSNE
import time
import sklearn
from sklearn.decomposition import PCA 
import librosa
from sklearn import mixture
from numpy import unique
from numpy import where
from matplotlib import pyplot as plt
import librosa.display
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.model_selection import GridSearchCV
from kneed import DataGenerator, KneeLocator
%matplotlib inline
import os, shutil, glob, os.path
from PIL import Image as pil_image
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.neighbors import kneighbors_graph

from numpy import load,save
from scipy.spatial import distance
from scipy.cluster import hierarchy
import umap

In [2]:
def load_filepaths():
    imdir_ideology = 'ideology_image_dataset/'
    imdir_muslim='muslim_image_dataset/'
    ideology_files=os.listdir('ideology_image_dataset/')
    muslim_files=os.listdir('muslim_image_dataset/')
    len(ideology_files),len(muslim_files)

    ideology_files_path=[os.path.join(imdir_ideology,file) for file in ideology_files ]
    muslim_files_path=[os.path.join(imdir_muslim,file) for file in muslim_files]
    return ideology_files_path,muslim_files_path

def loadFeatures(filename):
    print("Loading file : ",filename)
    features= load(filename)
    return features
    

In [3]:
def cv_silhouette_scorer(estimator, X):
    estimator.fit(X)
    try:
        cluster_labels = estimator.labels_
    except Exception as e:
      #  print(e,estimator)
        cluster_labels=estimator.predict(X)
    num_labels = len(set(cluster_labels))
    no_noise_labels=num_labels
    if(set(cluster_labels).issuperset({-1})):
        n_clustersLen=len(set(cluster_labels))-1
        no_noise_labels=n_clustersLen
    
    num_samples = len(X)
    if num_labels == 1 or num_labels == num_samples or no_noise_labels==1:
        return -1
    else:
        return metrics.silhouette_score(X, cluster_labels)



In [4]:
def evaluation_Score(features,y_pred,output_df,model):
    try:
        
        num_labels=len(set(y_pred))
        total_samples=len(y_pred)
        if(num_labels==1 or num_labels==total_samples):
            output_df.loc[model,'silhouette'] =-1
            output_df.loc[model,'calinski'] =-1
            output_df.loc[model,'davies'] =-1
            
        else:
            output_df.loc[model,'silhouette'] =metrics.silhouette_score(features,y_pred)
            output_df.loc[model,'calinski'] =metrics.calinski_harabasz_score(features, y_pred)
            output_df.loc[model,'davies'] =metrics.davies_bouldin_score(features,y_pred)
            
    

    except Exception as e:
        print(e)
        pass
        

        
    
    return output_df
    
    
    
    
   
    

In [5]:
def runGridSearch(estimator,params_dict,train_data):
    
    cv = [(slice(None), slice(None))]
    gs = GridSearchCV(estimator=estimator, param_grid=params_dict, scoring=cv_silhouette_scorer, cv=cv, n_jobs=-1)
    gs.fit(train_data)
    try:
        predicted_labels= gs.best_estimator_.labels_
    except:
        predicted_labels=gs.predict(train_data)
    
    print("best estimator is ",gs.best_estimator_)
    return predicted_labels

In [6]:


def runModels(train_data,output_df,filename):
   
   
        
    print("Agglomerative clustering results")
    #knn_graph = kneighbors_graph(train_data, 1, include_self=False)
   # grid_graph = grid_to_graph(*train_data.shape)
    params_dict={'linkage':['ward','average'],'distance_threshold':[5,6,10,20,30,40,50,60,70],'n_clusters':[None]}
    predicted_labels=runGridSearch(sklearn.cluster.AgglomerativeClustering(),params_dict,train_data)
    
    evaluation_Score(train_data,predicted_labels,output_df,'Agglomerative clustering')
    if(set(predicted_labels).issuperset({-1})):
        n_clustersLen=len(set(predicted_labels))-1
    else:
        n_clustersLen=len(set(predicted_labels))
    output_df.loc['Agglomerative clustering','n_clusters']=n_clustersLen
    
    np.save(filename+'_agg_labels.npy', np.array(predicted_labels))
    print(output_df)
    
    print("Birch")
    threshold_estimation=train_data.shape[1]/10
    if(threshold_estimation-10>=10):
        threshold_estimation-=10
    params_dict={'threshold':np.arange(threshold_estimation,threshold_estimation+5,0.3).tolist(),'n_clusters':[None]}
    predicted_labels=runGridSearch(sklearn.cluster.Birch(),params_dict,train_data)
    #print("number of distinct labels in the birch os")
    evaluation_Score(train_data,predicted_labels,output_df,'Birch')
    if(set(predicted_labels).issuperset({-1})):
        n_clustersLen=len(set(predicted_labels))-1
    else:
        n_clustersLen=len(set(predicted_labels))
    output_df.loc['Birch','n_clusters']=n_clustersLen
    
    np.save(filename+'_birch_labels.npy', np.array(predicted_labels))
    print(output_df)
    

   
    
   
    
    

    
   
    return output_df




In [7]:
def runClustering(train_data,filename):
   
    output_df = pd.DataFrame(index=['Birch','Agglomerative clustering'],columns=['n_clusters','silhouette','calinski'])
   
    output_df=runModels(train_data,output_df,filename)
    
 
   
    return output_df
        
    

In [ ]:
if __name__ == "__main__":
    
    ideology_files_path,muslim_files_path=load_filepaths()
    for file in os.listdir('Image_features/'):
        if file.endswith('.npy'):
            print(f'Image_features/{file}')
            features=loadFeatures(os.path.join('Image_features',file))
            print("tsne....")
            tsne_transformed=TSNE(n_components=3, n_jobs=-1).fit_transform(features)
            output_df=runClustering(tsne_transformed,f'image-results-hier/{file}-tsne')
            output_df.to_csv(f'image-results-hier/{file}-tsne.csv')
            
            
            
            print("pca....")
            pca_dims = PCA().fit(features)
            cumsum = np.cumsum(pca_dims.explained_variance_ratio_)
            d = np.argmax(cumsum >= 0.95) + 1
            print(d)
            if(d==1):
                d=d+1
            pca_transformed=PCA(n_components=d).fit_transform(features)
            output_df=runClustering(pca_transformed,f'image-results-hier/{file}-pca')
            output_df.to_csv(f'image-results-hier/{file}-pca.csv')
            
            print("umap....")
            reducer = umap.UMAP(random_state=42,n_components=2)
            umap_transformed = reducer.fit_transform(features)
            output_df=runClustering(umap_transformed,f'image-results-hier/{file}-umap')
            output_df.to_csv(f'image-results-hier/{file}-umap.csv')
            
            
            pca_50 = PCA(n_components=50)
            pca_result_50 = pca_50.fit_transform(features)
            tsne_transformed = TSNE(random_state = 42, n_components=3,verbose=0, perplexity=40, n_iter=400).fit_transform(pca_result_50)
            output_df=runClustering(tsne_transformed,f'image-results-hier/{file}-pca-tsne')
            output_df.to_csv(f'image-results-hier/{file}-pca-tsne.csv')
            
